In [1]:
import os
from pathlib import Path
project_root = os.path.join(str(Path.home()) + '/Documents', 'PPGADev')
os.chdir(project_root)
%pwd # should be PPGA root dir

'/home/icaros/Documents/PPGADev'

In [2]:
import pickle

import numpy as np
from attrdict import AttrDict
from RL.ppo import *
from utils.utilities import log
from envs.brax_custom.brax_env import make_vec_env_brax
from models.actor_critic import Actor, PGAMEActor
from pandas import DataFrame

from IPython.display import HTML, Image
from IPython.display import display
from brax.io import html, image

In [3]:
# params to config
device = torch.device('cuda')
# env_name = 'humanoid'
env_name = 'walker2d'
seed = 1111
normalize_obs = True
normalize_rewards = True
# non-configurable params
obs_shapes = {
    'humanoid': (227,),
    'ant': (87,),
    'halfcheetah': (18,),
    'walker2d': (17,)
}
action_shapes = {
    'humanoid': (17,),
    'ant': (8,),
    'halfcheetah': (6,),
    'walker2d': (6,)
}

# define the final config objects
actor_cfg = AttrDict({
        'obs_shape': obs_shapes[env_name],
        'action_shape': action_shapes[env_name],
        'normalize_obs': normalize_obs,
        'normalize_rewards': normalize_rewards,
})
env_cfg = AttrDict({
        'env_name': env_name,
        'env_batch_size': None,
        'num_dims': 2 if not 'ant' in env_name else 4,
        'envs_per_model': 1,
        'seed': seed,
        'num_envs': 1,
        'clip_obs_rew': False,
        'is_energy_measures': True
})

env_cfg_std = AttrDict({
        'env_name': env_name,
        'env_batch_size': None,
        'num_dims': 2 if not 'ant' in env_name else 4,
        'envs_per_model': 1,
        'seed': seed,
        'num_envs': 1,
        'clip_obs_rew': False,
        'is_energy_measures': False
})

if env_cfg.is_energy_measures:
    extra = 1 if not 'walker2d' in env_name else 2
    env_cfg.num_dims += extra


In [4]:
# now lets load in a saved archive dataframe and scheduler
# change this to be your own checkpoint path
# archive_path = 'experiments/paper_ppga_ant/1111/checkpoints/cp_00001390/archive_df_00001390.pkl'
# scheduler_path = 'experiments/paper_ppga_ant/1111/checkpoints/cp_00001390/scheduler_00001390.pkl'

# archive_path = 'experiments/paper_ppga_walker2d/1111/checkpoints/cp_00001700/archive_df_00001700.pkl'
# scheduler_path = 'experiments/paper_ppga_walker2d/1111/checkpoints/cp_00001700/scheduler_00001700.pkl'

archive_path = '/home/icaros/Documents/PPGADev/archive_df_00001420.pkl'
standard_archive_path = '/home/icaros/Documents/PPGADev/archive_df_00002000.pkl'
# scheduler_path = 'experiments/paper_ppga_humanoid/1111/checkpoints/cp_00002000/scheduler_00002000.pkl'

with open(archive_path, 'rb') as f:
    archive_df = pickle.load(f)
with open(standard_archive_path, 'rb') as f:
    standard_archive_df = pickle.load(f)
# with open(scheduler_path, 'rb') as f:
#     scheduler = pickle.load(f)

In [5]:
from ribs.archives import GridArchive
all_solutions = archive_df.solution_batch()
all_objectives = archive_df.objective_batch()
all_measures = archive_df.measures_batch()
all_metadata = archive_df.metadata_batch()
print(all_solutions.shape)

archive = GridArchive(
    solution_dim=all_solutions.shape[1],  # Dimensionality of solutions in the archive.
    dims=[20, 20, 20, 20],  # 50 cells along each dimension.
    ranges=[(0.0, 1.0), (0.0, 1.0), (0.0, 6.0), (0.0, 3.0)],  # (-1, 1) for x-pos and (-3, 0) for y-vel.
    qd_offset=0.0,  # See the note below.
)

standard_all_solutions = standard_archive_df.solution_batch()
standard_all_objectives = standard_archive_df.objective_batch()
standard_all_measures = standard_archive_df.measures_batch()
standard_all_metadata = standard_archive_df.metadata_batch()
print(standard_all_solutions.shape)

standard_archive = GridArchive(
    solution_dim=all_solutions.shape[1],  # Dimensionality of solutions in the archive.
    dims=[50,50],  # 50 cells along each dimension.
    ranges=[(0.0, 1.0), (0.0, 1.0)],  # (-1, 1) for x-pos and (-3, 0) for y-vel.
    qd_offset=0.0,  # See the note below.
)

# ant offset = 3.24, 5 dims - 4 (0, 1.0), 1 (0, 8.0), grid 10x10x10....
# walker offset = 1.413, 3 dims - 2 (0, 1.0), 1 (0, 6.0), grid 50x50x50
# humanoid offset = 0.0, 3 dims - 2 (0, 1.0), 1 (0, 16.0) grid 20x20x20

# archive.add(all_solutions, all_objectives, all_measures)

(8423, 19596)
(2072, 19596)


In [6]:
archive.add(all_solutions[0:1000], all_objectives[0:1000], all_measures[0:1000], all_metadata[0:1000])
standard_archive.add(standard_all_solutions[0:1000], 
                     standard_all_objectives[0:1000], 
                     standard_all_measures[0:1000], 
                     standard_all_metadata[0:1000])

(array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [7]:
i = 1001
end = min(i + 1000, all_solutions.shape[0])
while i < all_solutions.shape[0]:
    end = min(i + 1000, all_solutions.shape[0])
    archive.add(all_solutions[i:end], all_objectives[i:end], all_measures[i:end], all_metadata[i:end])
    print(i, " to ", end)
    i += 1000
    
i = 1001
end = min(i + 1000, standard_all_solutions.shape[0])
while i < standard_all_solutions.shape[0]:
    end = min(i + 1000, standard_all_solutions.shape[0])
    standard_archive.add(standard_all_solutions[i:end], 
                         standard_all_objectives[i:end], 
                         standard_all_measures[i:end], 
                         standard_all_metadata[i:end])
    print(i, " to ", end)
    i += 1000
    

1001  to  2001
2001  to  3001
3001  to  4001
4001  to  5001
5001  to  6001
6001  to  7001
7001  to  8001
8001  to  8423
1001  to  2001
2001  to  2072


In [8]:
# create the environment

print("env")
env = make_vec_env_brax(env_cfg)

env
feet contact, is_energy_measures =  True
env step fn
<function walker2d_step at 0x7f414d9c91f0>
feet contact and energy


In [9]:
def get_best_elite(standard_ppga=False):
    if standard_ppga:
        best_elite = standard_archive.best_elite
    else:
        best_elite = archive.best_elite
    print(f'Loading agent with reward {best_elite.objective} and measures {best_elite.measures}')
    agent = Actor(obs_shape=actor_cfg.obs_shape[0], action_shape=actor_cfg.action_shape, normalize_obs=normalize_obs, normalize_returns=normalize_rewards).deserialize(best_elite.solution).to(device)
#     print("elite metadata")
#     print(best_elite.metadata)
    if actor_cfg.normalize_obs:
        norm = best_elite.metadata['obs_normalizer']
        if isinstance(norm, dict):
            agent.obs_normalizer.load_state_dict(norm)
        else:
            agent.obs_normalizer = norm
    return agent

In [10]:
def get_random_elite(standard_ppga=False):
    if standard_ppga:
        elite = standard_archive.sample_elites(1)
    else:
        elite = archive.sample_elites(1)

    print(f'Loading agent with reward {elite.objective[0]} and measures {elite.measures[0]}')
    agent = Actor(obs_shape=actor_cfg.obs_shape[0], action_shape=actor_cfg.action_shape, normalize_obs=normalize_obs, normalize_returns=normalize_rewards).deserialize(elite.solution_batch.flatten()).to(device)
    if actor_cfg.normalize_obs:
        norm = elite.metadata['obs_normalizer']
        if isinstance(norm, dict):
            agent.obs_normalizer.load_state_dict(norm)
        else:
            agent.obs_normalizer = norm
    return agent

In [11]:
def get_elite(measures, standard_ppga=False):
#     elite = scheduler.archive.elites_with_measures_single(measures)

    if standard_ppga:
        elite = standard_archive.elites_with_measures_single(measures)
    else:
        elite = archive.elites_with_measures_single(measures)
    elite = archive.elites_with_measures_single(measures)
    print(f'Loading agent with reward {elite.objective} and measures {elite.measures}')
    agent = Actor(obs_shape=actor_cfg.obs_shape[0], action_shape=actor_cfg.action_shape, normalize_obs=normalize_obs, normalize_returns=normalize_rewards).deserialize(elite.solution.flatten()).to(device)
    print("elite")
    print(elite)
#     print("elite metadata")
#     print(elite.metadata)
    if actor_cfg.normalize_obs:
        norm = elite.metadata['obs_normalizer']
        if isinstance(norm, dict):
            agent.obs_normalizer.load_state_dict(norm)
        else:
            agent.obs_normalizer = norm
    return agent

In [12]:
def enjoy_brax(agent, render=True, deterministic=True, standard_ppga=False):
    if actor_cfg.normalize_obs:
        obs_mean, obs_var = agent.obs_normalizer.obs_rms.mean, agent.obs_normalizer.obs_rms.var
        print(f'{obs_mean=}, {obs_var=}')

    obs = env.reset()
    rollout = [env.unwrapped._state]
    total_reward = 0
    measures = torch.zeros(env_cfg.num_dims).to(device)
#     if standard_ppga:
#         measures = torch.zeros(env_cfg_std.num_dims).to(device)
#     else:
#         measures = torch.zeros(env_cfg.num_dims).to(device)
    done = False
    while not done:
        with torch.no_grad():
            obs = obs.unsqueeze(dim=0).to(device)
            if actor_cfg.normalize_obs:
                obs = (obs - obs_mean) / torch.sqrt(obs_var + 1e-8)

            if deterministic:
                act = agent.actor_mean(obs)
            else:
                act, _, _ = agent.get_action(obs)
            act = act.squeeze()
            obs, rew, done, info = env.step(act.cpu())
#             print("measures.shape = ", measures.shape)
#             print("info['measures'].shape = ", info['measures'].shape)
            measures += info['measures']
            rollout.append(env.unwrapped._state)
            total_reward += rew
    if render:
        i = HTML(html.render(env.unwrapped._env.sys, [s.qp for s in rollout]))
        display(i)
        print(f'{total_reward=}')
        print(f' Rollout length: {len(rollout)}')
        measures /= len(rollout)
        print(f'Measures: {measures.cpu().numpy()}')
    return total_reward.detach().cpu().numpy()

In [13]:
# agent = get_random_elite()
print("line")
print("line")
agent = get_best_elite(standard_ppga=True)
enjoy_brax(agent, render=True, deterministic=True, standard_ppga=True)

line
line
Loading agent with reward 4340.559712511301 and measures [0.22344469 0.33704257]
obs_mean=tensor([ 1.0632, -0.5663, -0.4077, -0.8044,  0.2120, -0.7952, -0.5055,  0.4120,
         1.6020, -0.1479, -0.1618,  0.1887,  0.3749,  0.0059,  0.4005,  0.2628,
        -0.1134], device='cuda:0'), obs_var=tensor([0.0137, 0.0937, 0.2266, 0.1764, 0.2296, 0.2683, 0.3901, 0.2303, 1.6711,
        1.0290, 2.1029, 4.8919, 6.4756, 9.2637, 4.9492, 4.8698, 5.3145],
       device='cuda:0')
inside walker2d, add z height to ctrl cost measure


total_reward=tensor(3650.3352, device='cuda:0')
 Rollout length: 1001
Measures: [0.2027972 0.2937063 4.792021  1.0694517]


array(3650.3352, dtype=float32)

In [14]:
# agent = get_random_elite()
print("line")
print("line")
agent = get_best_elite(standard_ppga=False)
enjoy_brax(agent, render=True, deterministic=True, standard_ppga=False)

line
line
Loading agent with reward 2860.608291817093 and measures [0.30239999 0.25750002 5.2389636  0.74023563]
obs_mean=tensor([ 0.4286,  0.4512, -0.7119, -0.6987,  0.2361, -0.9150, -0.7200,  0.3461,
         0.4157, -0.1925,  0.6697,  0.1832,  0.1683, -0.0170,  0.2382,  0.1692,
        -0.0502], device='cuda:0'), obs_var=tensor([0.1807, 0.9606, 0.8632, 0.8269, 0.3397, 1.0682, 0.8929, 0.3292, 0.5358,
        0.5807, 2.6056, 2.0187, 2.7948, 2.5940, 2.0481, 2.5704, 2.8775],
       device='cuda:0')


total_reward=tensor(1382.3971, device='cuda:0')
 Rollout length: 1001
Measures: [0.4035964  0.3936064  5.027602   0.76684314]


array(1382.3971, dtype=float32)

In [15]:
agent2 = get_elite([0.20, 0.29, 4.79, 1.07])
print(agent2)
enjoy_brax(agent2, render=True, deterministic=True)

Loading agent with reward nan and measures [nan nan nan nan]
elite
Elite(solution=array([nan, nan, nan, ..., nan, nan, nan]), objective=nan, measures=array([nan, nan, nan, nan]), index=-1, metadata=None)


TypeError: 'NoneType' object is not subscriptable

In [ ]:
import matplotlib.pyplot as plt
from ribs.visualize import parallel_axes_plot
plt.figure(figsize=(8, 6))
parallel_axes_plot(archive)
plt.show

In [ ]:
# visualizing archive - ANT

In [ ]:
# ant
from ribs.archives import CVTArchive
cvt_archive1 = CVTArchive(
    solution_dim=all_solutions.shape[1],  # Dimensionality of solutions in the archive.
    cells=1000,
    ranges=[(0.0, 1.0), (0.0, 8.0)]
)

cvt_archive2 = CVTArchive(
    solution_dim=all_solutions.shape[1],  # Dimensionality of solutions in the archive.
    cells=1000,
    ranges=[(0.0, 1.0), (0.0, 8.0)]
)

cvt_archive3 = CVTArchive(
    solution_dim=all_solutions.shape[1],  # Dimensionality of solutions in the archive.
    cells=1000,
    ranges=[(0.0, 1.0), (0.0, 8.0)]
)

cvt_archive4 = CVTArchive(
    solution_dim=all_solutions.shape[1],  # Dimensionality of solutions in the archive.
    cells=1000,
    ranges=[(0.0, 1.0), (0.0, 8.0)]
)

cvt_archive1.add(all_solutions[0:1000], all_objectives[0:1000], all_measures[0:1000][:, [0, 4]], all_metadata[0:1000])
cvt_archive2.add(all_solutions[0:1000], all_objectives[0:1000], all_measures[0:1000][:, [1, 4]], all_metadata[0:1000])
cvt_archive3.add(all_solutions[0:1000], all_objectives[0:1000], all_measures[0:1000][:, [2, 4]], all_metadata[0:1000])
cvt_archive4.add(all_solutions[0:1000], all_objectives[0:1000], all_measures[0:1000][:, [3, 4]], all_metadata[0:1000])


In [ ]:
i = 1001
end = min(i + 1000, all_solutions.shape[0])
while i < all_solutions.shape[0]:
    end = min(i + 1000, all_solutions.shape[0])
#     archive.add(all_solutions[i:end], all_objectives[i:end], all_measures[i:end], all_metadata[i:end])

    cvt_archive1.add(all_solutions[i:end], all_objectives[i:end], all_measures[i:end][:, [0, 4]], all_metadata[i:end])
    cvt_archive2.add(all_solutions[i:end], all_objectives[i:end], all_measures[i:end][:, [1, 4]], all_metadata[i:end])
    cvt_archive3.add(all_solutions[i:end], all_objectives[i:end], all_measures[i:end][:, [2, 4]], all_metadata[i:end])
    cvt_archive4.add(all_solutions[i:end], all_objectives[i:end], all_measures[i:end][:, [3, 4]], all_metadata[i:end])
    print(i, " to ", end)
    i += 1000


In [ ]:
import matplotlib.pyplot as plt
from ribs.visualize import cvt_archive_heatmap
cvt_archive_heatmap(cvt_archive1)
plt.title("Measure 1 vs Energy")
plt.show()
cvt_archive_heatmap(cvt_archive2)
plt.title("Measure 2 vs Energy")
plt.show()
cvt_archive_heatmap(cvt_archive3)
plt.title("Measure 3 vs Energy")
plt.show()
cvt_archive_heatmap(cvt_archive4)
plt.title("Measure 4 vs Energy")
plt.show()

In [ ]:
# ant
# from ribs.archives import CVTArchive
cvt_archive12 = CVTArchive(
    solution_dim=all_solutions.shape[1],  # Dimensionality of solutions in the archive.
    cells=1000,
    ranges=[(0.0, 1.0), (0.0, 1.0)]
)

cvt_archive13 = CVTArchive(
    solution_dim=all_solutions.shape[1],  # Dimensionality of solutions in the archive.
    cells=1000,
    ranges=[(0.0, 1.0), (0.0, 1.0)]
)

cvt_archive14 = CVTArchive(
    solution_dim=all_solutions.shape[1],  # Dimensionality of solutions in the archive.
    cells=1000,
    ranges=[(0.0, 1.0), (0.0, 1.0)]
)

cvt_archive23 = CVTArchive(
    solution_dim=all_solutions.shape[1],  # Dimensionality of solutions in the archive.
    cells=1000,
    ranges=[(0.0, 1.0), (0.0, 1.0)]
)


cvt_archive24 = CVTArchive(
    solution_dim=all_solutions.shape[1],  # Dimensionality of solutions in the archive.
    cells=1000,
    ranges=[(0.0, 1.0), (0.0, 1.0)]
)


cvt_archive34 = CVTArchive(
    solution_dim=all_solutions.shape[1],  # Dimensionality of solutions in the archive.
    cells=1000,
    ranges=[(0.0, 1.0), (0.0, 1.0)]
)


cvt_archive12.add(all_solutions[0:1000], all_objectives[0:1000], all_measures[0:1000][:, [0, 1]], all_metadata[0:1000])
cvt_archive13.add(all_solutions[0:1000], all_objectives[0:1000], all_measures[0:1000][:, [0, 2]], all_metadata[0:1000])
cvt_archive14.add(all_solutions[0:1000], all_objectives[0:1000], all_measures[0:1000][:, [0, 3]], all_metadata[0:1000])
cvt_archive23.add(all_solutions[0:1000], all_objectives[0:1000], all_measures[0:1000][:, [1, 2]], all_metadata[0:1000])
cvt_archive24.add(all_solutions[0:1000], all_objectives[0:1000], all_measures[0:1000][:, [1, 3]], all_metadata[0:1000])
cvt_archive34.add(all_solutions[0:1000], all_objectives[0:1000], all_measures[0:1000][:, [2, 3]], all_metadata[0:1000])


In [ ]:
i = 1001
end = min(i + 1000, all_solutions.shape[0])
while i < all_solutions.shape[0]:
    end = min(i + 1000, all_solutions.shape[0])
#     archive.add(all_solutions[i:end], all_objectives[i:end], all_measures[i:end], all_metadata[i:end])

    cvt_archive12.add(all_solutions[i:end], all_objectives[i:end], all_measures[i:end][:, [0, 1]], all_metadata[i:end])
    cvt_archive13.add(all_solutions[i:end], all_objectives[i:end], all_measures[i:end][:, [0, 2]], all_metadata[i:end])
    cvt_archive14.add(all_solutions[i:end], all_objectives[i:end], all_measures[i:end][:, [0, 3]], all_metadata[i:end])
    cvt_archive23.add(all_solutions[i:end], all_objectives[i:end], all_measures[i:end][:, [1, 2]], all_metadata[i:end])
    cvt_archive24.add(all_solutions[i:end], all_objectives[i:end], all_measures[i:end][:, [1, 3]], all_metadata[i:end])
    cvt_archive34.add(all_solutions[i:end], all_objectives[i:end], all_measures[i:end][:, [2, 3]], all_metadata[i:end])
    print(i, " to ", end)
    i += 1000


In [ ]:

cvt_archive_heatmap(cvt_archive12)
plt.title("Measure 1 vs Measure 2")
plt.show()
cvt_archive_heatmap(cvt_archive13)
plt.title("Measure 1 vs Measure 3")
plt.show()
cvt_archive_heatmap(cvt_archive14)
plt.title("Measure 1 vs Measure 4")
plt.show()
cvt_archive_heatmap(cvt_archive23)
plt.title("Measure 2 vs Measure 3")
plt.show()
cvt_archive_heatmap(cvt_archive24)
plt.title("Measure 2 vs Measure 4")
plt.show()
cvt_archive_heatmap(cvt_archive34)
plt.title("Measure 3 vs Measure 4")
plt.show()

In [ ]:
# visualizing archive - WALKER2D

In [ ]:
# visualizing archive - WALKER2D
# walker2d     ranges=[(0.0, 1.0), (0.0, 1.0), (0.0, 6.0), (0.0, 3.0)],  # (-1, 1) for x-pos and (-3, 0) for y-vel.
from ribs.archives import CVTArchive
cvt_archive1 = CVTArchive( #measure 1 vs energy
    solution_dim=all_solutions.shape[1],  # Dimensionality of solutions in the archive.
    cells=1000,
    ranges=[(0.0, 1.0), (0.0, 6.0)]
)

cvt_archive2 = CVTArchive( #measure 2 vs energy
    solution_dim=all_solutions.shape[1],  # Dimensionality of solutions in the archive.
    cells=1000,
    ranges=[(0.0, 1.0), (0.0, 6.0)]
)

cvt_archive12 = CVTArchive( #measure 1 vs measure 2
    solution_dim=all_solutions.shape[1],  # Dimensionality of solutions in the archive.
    cells=1000,
    ranges=[(0.0, 1.0), (0.0, 1.0)]
)


cvt_archive1z = CVTArchive( #measure 1 vs z height
    solution_dim=all_solutions.shape[1],  # Dimensionality of solutions in the archive.
    cells=1000,
    ranges=[(0.0, 1.0), (0.0, 3.0)]
)


cvt_archive2z = CVTArchive( #measure 2 vs z height
    solution_dim=all_solutions.shape[1],  # Dimensionality of solutions in the archive.
    cells=1000,
    ranges=[(0.0, 1.0), (0.0, 3.0)]
)


cvt_archiveez = CVTArchive( #energy vs z height
    solution_dim=all_solutions.shape[1],  # Dimensionality of solutions in the archive.
    cells=1000,
    ranges=[(0.0, 6.0), (0.0, 3.0)]
)

print("cvt archives created, loading solutions...")

cvt_archive1.add(all_solutions[0:1000], all_objectives[0:1000], all_measures[0:1000][:, [0, 2]], all_metadata[0:1000])
cvt_archive2.add(all_solutions[0:1000], all_objectives[0:1000], all_measures[0:1000][:, [1, 2]], all_metadata[0:1000])
cvt_archive12.add(all_solutions[0:1000], all_objectives[0:1000], all_measures[0:1000][:, [0, 1]], all_metadata[0:1000])

cvt_archive1z.add(all_solutions[0:1000], all_objectives[0:1000], all_measures[0:1000][:, [0, 3]], all_metadata[0:1000])
cvt_archive2z.add(all_solutions[0:1000], all_objectives[0:1000], all_measures[0:1000][:, [1, 3]], all_metadata[0:1000])
cvt_archiveez.add(all_solutions[0:1000], all_objectives[0:1000], all_measures[0:1000][:, [2, 3]], all_metadata[0:1000])


In [ ]:
i = 1001
end = min(i + 1000, all_solutions.shape[0])
while i < all_solutions.shape[0]:
    end = min(i + 1000, all_solutions.shape[0])
#     archive.add(all_solutions[i:end], all_objectives[i:end], all_measures[i:end], all_metadata[i:end])

    cvt_archive1.add(all_solutions[i:end], all_objectives[i:end], all_measures[i:end][:, [0, 2]], all_metadata[i:end])
    cvt_archive2.add(all_solutions[i:end], all_objectives[i:end], all_measures[i:end][:, [1, 2]], all_metadata[i:end])
    cvt_archive12.add(all_solutions[i:end], all_objectives[i:end], all_measures[i:end][:, [0, 1]], all_metadata[i:end])
        
    cvt_archive1z.add(all_solutions[i:end], all_objectives[i:end], all_measures[i:end][:, [0, 3]], all_metadata[i:end])
    cvt_archive2z.add(all_solutions[i:end], all_objectives[i:end], all_measures[i:end][:, [1, 3]], all_metadata[i:end])
    cvt_archiveez.add(all_solutions[i:end], all_objectives[i:end], all_measures[i:end][:, [2, 3]], all_metadata[i:end])
    print(i, " to ", end)
    i += 1000


In [ ]:
print("solutions added, plotting...")
import matplotlib.pyplot as plt
from ribs.visualize import cvt_archive_heatmap
cvt_archive_heatmap(cvt_archive1)
plt.title("Measure 1 vs Energy")
plt.show()
# plt.savefig(exp_dir + 'm1_vs_energy.png')
cvt_archive_heatmap(cvt_archive2)
plt.title("Measure 2 vs Energy")
plt.show()
# plt.savefig(exp_dir + 'm2_vs_energy.png')
cvt_archive_heatmap(cvt_archive12)
plt.title("Measure 1 vs Measure 2")
plt.show()
# plt.savefig(exp_dir + 'm1_vs_m2.png')
cvt_archive_heatmap(cvt_archive1z)
plt.title("Measure 1 vs Z")
plt.show()
# plt.savefig(exp_dir + 'm1_vs_z.png')
cvt_archive_heatmap(cvt_archive2z)
plt.title("Measure 2 vs Z")
plt.show()
# plt.savefig(exp_dir + 'm2_vs_z.png')
cvt_archive_heatmap(cvt_archiveez)
plt.title("Energy vs Z")
plt.show()
# plt.savefig(exp_dir + 'energy_vs_z.png')

print("done")


In [ ]:
# visualizing archive - HUMANOID

In [ ]:
# humanoid
from ribs.archives import CVTArchive
cvt_archive1 = CVTArchive(
    solution_dim=all_solutions.shape[1],  # Dimensionality of solutions in the archive.
    cells=1000,
    ranges=[(0.0, 1.0), (0.0, 16.0)]
)

cvt_archive2 = CVTArchive(
    solution_dim=all_solutions.shape[1],  # Dimensionality of solutions in the archive.
    cells=1000,
    ranges=[(0.0, 1.0), (0.0, 16.0)]
)

cvt_archive12 = CVTArchive(
    solution_dim=all_solutions.shape[1],  # Dimensionality of solutions in the archive.
    cells=1000,
    ranges=[(0.0, 1.0), (0.0, 1.0)]
)

cvt_archive1.add(all_solutions[0:1000], all_objectives[0:1000], all_measures[0:1000][:, [0, 2]], all_metadata[0:1000])
cvt_archive2.add(all_solutions[0:1000], all_objectives[0:1000], all_measures[0:1000][:, [1, 2]], all_metadata[0:1000])
cvt_archive12.add(all_solutions[0:1000], all_objectives[0:1000], all_measures[0:1000][:, [0, 1]], all_metadata[0:1000])

In [ ]:
i = 1001
end = min(i + 1000, all_solutions.shape[0])
while i < all_solutions.shape[0]:
    end = min(i + 1000, all_solutions.shape[0])
#     archive.add(all_solutions[i:end], all_objectives[i:end], all_measures[i:end], all_metadata[i:end])

    cvt_archive1.add(all_solutions[i:end], all_objectives[i:end], all_measures[i:end][:, [0, 2]], all_metadata[i:end])
    cvt_archive2.add(all_solutions[i:end], all_objectives[i:end], all_measures[i:end][:, [1, 2]], all_metadata[i:end])
    cvt_archive12.add(all_solutions[i:end], all_objectives[i:end], all_measures[i:end][:, [0, 1]], all_metadata[i:end])
    print(i, " to ", end)
    i += 1000


In [ ]:
import matplotlib.pyplot as plt
from ribs.visualize import cvt_archive_heatmap
cvt_archive_heatmap(cvt_archive1)
plt.title("Measure 1 vs Energy")
plt.show()
cvt_archive_heatmap(cvt_archive2)
plt.title("Measure 2 vs Energy")
plt.show()
cvt_archive_heatmap(cvt_archive12)
plt.title("Measure 1 vs Measure 2")
plt.show()